In [1]:
import sys
import os

import Cassiopeia.ProcessingPipeline.process as process
import Cassiopeia.ProcessingPipeline.process.sequencing as sequencing
import pandas as pd 

import Cassiopeia
import numpy as np

import tqdm


## Sometimes Jupyter notebook doesn't retain your PATH environment variable -- this will mess up a number of things.
## We recommend specifying the environment variable manually here
os.environ["PATH"] = '/home/mattjones/bin:/home/mattjones/.local/bin:/home/mattjones/myapps/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/mattjones/opt/gurobi801/linux64/bin:/home/mattjones/software/bowtie2-2.3.4.2:/home/mattjones/emboss/EMBOSS-6.6.0/emboss'

In [ ]:
## first specify the home directory, and possorted genome bam
home_dir = "."
genome_bam = "data/test_possorted_genome_bam.bam"


### Collapsing UMI reads together

The first step of the pipeline is to collapse UMIs -- i.e. count how many times we read the sequence of each UMI. We also account for small sequencing errors by aggregating counts of UMIs whose sequences are very similar to one another. 

This function takes in the following parameters:

- **home_dir**: the filepath to the cellranger directory, storing the possorted bam
- **fn**: the path to the genome bam file
- **max_indels**: the maximum number of indels allowed between sequences for collapsing
- **max_hq_mismatches**: the maximum number of hamming distance between sequences for collapsing
- **max_UMI_distance**: the maximum number of mismatches between UMI sequences for collapsing
- **show_progress**: whether or not to show the tqdm progress bar
- **force_sort**: sort the input bam

This function can also be run via command line with the `collapse` command -- use `collapse -h` to view all options. 

Finally, this function will ouptut a collapsed .bam file of reads as well as a .txt file, storing all reads and meta information that can be read in using pandas dataframes.



In [ ]:
process.collapseUMIs(home_dir, genome_bam, force_sort=True)

### Pick Sequences

After running the collapsing function, which aggregates together identical reads, we'd like to assign a unique read to each UMI barcode sequence. The way we accomplish this is by simply selecting the sequence associated with the UMI that has the most reads. The majority of times, these will be clear; however, we still perform UMI filtering downstream that will remove any ambiguous reads. This is also the first step that performs a cell-barcode filtering, based on the number of UMIs that are observed per cell. 

The parameters that can be passed to this function are the following:

- **moltable**: the moleculetable dataframe on which to perform this function (this can be the `.collpased.txt` dataframe created in the last step)
- **out_fp**: the output path name of the resulting dataframe
- **output_dir**: the output directory for storing all plotting output
- **cell_umi_thresh**: the minimum number of UMIs per cell, used for cell filtering (default = 10)
- **avg_reads_per_UMI_thresh**: the number of reads per UMI allowed, used for UMI filtering (default = 2)
- **save_output**: allows for writing of the text file; else just the molecule table will be returned (default = False)

In [ ]:
process.pickSeq("data/test_possorted_genome_bam_sorted.collapsed.txt", "data/test_possorted_genome_bam.picked.txt", "data/", cell_umi_thresh=5, avg_reads_per_UMI_thresh=1, save_output=True)

### Aligning of Sequences & Calling Alleles

With individual mappings of sequences to UMIs, we're now ready to call the alleles for each UMI -- namely, the outcomes of Cas9 cuts that will be used to reconstruct lineages. We use the Smith-Waterman aligner here as it is good at removing parts of sequences that result from outrageous editing outcomes (e.g. translocations or complete removal of the target site). We find that the optimal parameters consist of a Gap Opening penalty of 20 and a Gap Extension Penalty of 1, as found via simulations (see the manuscript for more details). 

The function that should be used for this step is `align_sequences`, which takes the following parameters:

- **ref**: file path to the reference sequence
- **queries**: file path to the dataframe consisting of UMI sequences to be aligned 
- **outfile**: file path to the output file
- **gapopen**: gap open penalty (default = 20)
- **gapextend**: gap extension penalty (default = 1)
- **ref_format**: reference sequence format (default = fasta)
- **query_format**: query sequence foramt (default = fastq)
- **out_format**: output format (default = sam)


The result of this function will be the creation of an output sam file, which can be used to call alleles and then turned into a UMI-table, mapping UMIs to alleles.

In [ ]:
process.align_sequences("data/PCT48-long.ref.fa", "data/test_possorted_genome_bam.picked.txt", "data/test_sw_aligned.sam")

We now call indels by reading the alignment CIGAR string. This essentially consists of parsing out where insertions are deletions were detected from the alignment. We then output a file consisting of the indels called for each sequence; these will eventually be used to reconstruct lineages. 

The function used for this step ``call_indels`` takes the following parameters:

- **alignments**: a .sam file containing the alignments.
- **ref**: FASTA file containing the reference sequence for annotating where indels occurred & grabbing the context around them.
- **output**: Output file, to be written as a .sam file.
- **context**: Boolean value, indicating whether or not to include sequence context around the indel.

This function will write to output a .sam file containing the called indels for each sequence.

In [ ]:
process.call_indels("data/test_sw_aligned.sam", "data/PCT48-long.ref.fa", "data/test_umi_table.sam")

### Error Correct UMIs

We can now define equivalence classes of UMIs associated with idential cells and alelles. We'd like to error correct within these classes from UMIs with few reads to UMIs in the same equivalence class with high reads. This will overall reduce the amount of noise in the dataset and make more coherent alleletables for lineage group calling, doublet detection, and lineage reconstruction. 

You can use the `errorCorrectUMIs` function which takes the following parameters:

- **input_fn**: the bam file on which to error correct
- **_id**: the id to use 
- **log_file**: the file to log the error corrections in 
- **max_UMI_distance**: the maximum hamming distance between UMIs allowed (default = 2)
- **show_progress**: update progress with tqdm (default = True)

In [ ]:
process.errorCorrectUMIs("data/test_umi_table.bam", "test", "ec_log.txt")

### Filter Molecule Table

This module will filter out low-quality cells, perform error correction on the intBCs, and throw out any doublets that report conflicting indel information. This function will call the ``filter-molecule-table`` command via dispatch to command line.  This wrapper function takes the following parameters:

- **mt**: File path to the molecule table to be filtered
- **out_fp**: Output file name. This will be written ``outputdir``
- **outputdir**: Directory to output all logs and files
- **cell_umi_filter**: Cell UMI Threshold for filtering (Default = 10)
- **umi_read_thresh**: UMI read threshold for filtering (Default = dynamically computed using the 99th percentile / 10)
- **intbc_prop_thresh**: A minimum proportion of reads required for the more abundant intBC during intBC error correction. If the proportion is not met, then error correction is not performed (i.e. we cannot trust this more abundant intBC enough).
- **intbc_umi_thresh**: A minimum number of UMIs that need to b observed for the more abundant intBC during intBC error correction. 
- **intbc_dist_thresh**: A hamming distance threshold for considering intBCs to be error corrected
- **verbose**: Verbose output, consisting of output log files.
- **ec_intbc**: Error correct integration barcodes (Default = False)
- **detect_intra_doublets**: Detect doublets based on conflicting indel information (Default = True)
- **doublet_threshold**: Threshold to be used during intra doublet detection (Default = 0.35)

This file will write to file a filtered molecule table. 

In [ ]:
process.filter_molecule_table("data/test_umi_table_sorted_ec.moleculeTable.txt", "test.moleculeTable.filtered.txt", "data/", doublet_threshold=0.1)

If you have multiple samples, we have a small script that will append to the cellBC a sampleID of your choice. The parameters for this function, ``changeCellBCID`` are:

- **alleleTableIN**: allele table to have sampleIDs appended
- **sampleID**: sampleID to append to the cellBC
- **alleleTableOUT**: allele table filepath for the output

In [ ]:
process.changeCellBCID("data/test.moleculeTable.filtered.txt", "test", "data/test.moleculeTable.filtered.sample.txt")

### Call Lineage Groups

Our last step before reconstructing trees is to call "Lineage Groups" (these are clonal populations in our datasets, identifiable by their unique set of intBCs). The function here, ``call_lienage_groups`` is a wrapper for a command line tool that takes in an allele table that has already been filtered and divides the dataset into clonal populations. Here, we can also attempt to filter out doublets that consist of cells from different clones (based on their set of intBCs). The function takes the following parameters:

- **mt**: filepath to the filtered molecule table
- **out_fp**: output file path, to be written in the specified output directory
- **outputdir**: output directory
- **cell_umi_thresh**: Cell UMI Threshold (Default = 10)
- **min_cluster_prop**: lower bound of lineage group size, as defined as a proportion of the total number of cells. Given as a float between 0 and 1 (Default = 0.005). 
- **kinship_thresh**: Threshold on which to filter out cells from a lineage group during iterative assignment, based on the proportion of intBCs that a cell shares with that lineage group. Given as a float between 0 and 1 (Default = 0.25).
- **min_intbc_thresh**: Filtering criteria for intBCs at the lineage group level - the minimum proportion of cells that must hve an intBC for the intBC to be considered legitimate for reconstruction. (Default = 0.05)
- **detect_doublets_inter**: Perform inter doublet detection (Default = True)
- **doublet_threshold**: Threshold to use for doublet detection (0.35)
- **verbose**: Allow output of log files (Default = False)
- **plot**: Allow plotting at the end of the pipeline (Default = False)

This function will write an allele table with a new column, "lineageGrp",  denoting which clone each cell belongs to.

In [ ]:
process.call_lineage_groups("data/test.moleculeTable.filtered.sample.txt", "test.alleleTable.txt", "lg_output")

call-lineages data/test.moleculeTable.filtered.sample.txt test.alleleTable.txt lg_output --min_cluster_prop 0.005 --min_intbc_thresh 0.05 --doublet_threshold 0.35 --cell_umi_filter 10 --detect_doublets_inter
